In [18]:
from math import log
import numpy as np
import operator

## Function to calculate the Shannon entropy of a dataSet

In [5]:
def calcShannonEnt(dataSet):
    m = len(dataSet)
    labelCounts = {}
    for featVec in dataSet: # the number of unique elements and their occurance
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/m
        shannonEnt -= prob * log(prob, 2) # log base 2
    return shannonEnt

def createDataSet():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    #change to discrete values
    return dataSet, labels

In [10]:
dataSet, labels = createDataSet()
print('the Shannon Entropy of dataSet:',calcShannonEnt(dataSet))

the Shannon Entropy of dataSet: 0.9709505944546686


## choose and split the best feature

In [15]:
def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis] # chop out axis used for splitting
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

def chooseBestFeatureToSplit(dataSet):
    n = np.shape(dataSet)[1] - 1 # the last column is used for the labels
    baseEntropy = calcShannonEnt(dataSet)
    bestInforGain = 0.0; bestFeature = -1
    for i in range(n):   # iterate over all the features
        featList = [example[i] for example in dataSet] # create a list of all the examples of this feature
        uniqueVals = set(featList)
        newEntropy = 0.0
        for value  in uniqueVals: # Calculate entropy for each split
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet) / float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
        inforGain = baseEntropy - newEntropy # calculate the info gain; 
        if(inforGain > bestInforGain):  # compare this to the best gain so far
            bestInforGain = inforGain   # if better than current best, set to best
            bestFeature = i
    return bestFeature  # return an integer

In [17]:
chooseBestFeatureToSplit(dataSet)

0

In [19]:
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.key():
            classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

def createTree(dataSet, labels):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0]#stop splitting when all of the classes are equal
    if len(dataSet[0]) == 1: #stop splitting when there are no more features in dataSet
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]       #copy all of labels, so trees don't mess up existing labels
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value), subLabels)
    return myTree    

In [20]:
myTree = createTree(dataSet, labels)
myTree

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}